<a href="https://colab.research.google.com/github/Jaquetti/Automatic-annotation-YOLOv5/blob/main/Automatic_annotation_YOLOv5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import os
import time

**Este notebook utiliza a yolov5s treinada para auto-anotar imagens, sejam elas provenientes de conjunto de imagens em pastas ou diretamente de vídeos. Ao final as imagens são salvas em uma nova pasta junta ao seus txts correpondentes formatados padrão yolo.**

Lendo o modelo já treinado com uma confiança de 70%

In [ ]:
#Lendo modelo
model = torch.hub.load('/content/drive/MyDrive/yolov5_LIV/yolov5', 'custom', path='/content/drive/MyDrive/yolov5_LIV/yolov5/runs/train/exp148/weights/best.pt', source='local') 

#Modelo irá mostrar apenas predições com 70% de certeza
model.conf = 0.7

Auto-anotação de imagens provenientes de pastas

In [ ]:
#Setando o caminho onde a imagem está localizado
path_img = '/content/drive/MyDrive/d1/'

#Caminho onde os arquivos serão salvos
save_path ='/content/drive/MyDrive/d1_anno/'

try:
  os.mkdir(save_path)
  print('Criando o diretório')
except:
  print('Diretório encontrado')

#Carregando a imagem
#Variáveis para se utilizar no salvamento dos frames e anotações

#Flag que vai controlar a taxa de frames que serão processados 
frame_count = 0

#Rodando o loop equanto o vídeo não estiver finalizado 
for i in range(len(os.listdir(path_img))):
  #Lendo cada Frame
  frame = cv2.imread(path_img+os.listdir(path_img)[i])
 
  #Converten do tipo numpy array para PIL.image (aumenta a precisão por algum motivo)
  frame_pil = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
  frame_pil = Image.fromarray(frame_pil)

  #Enviando a imagem para a rede
  results = model(frame_pil,  size=1280)
        
  #Transformando as predições do modelo em formato pandas DataFrame 
  dados = results.pandas().xyxy[0]
  
  #Realizando o préprocessamento necessário para transformar as predições no 
  #formato padrão das anotações do modelo YOLO 
  im_w, im_h = frame_pil.size
  dados = dados[['class','xmin', 'ymin', 'xmax', 'ymax']]
  dados['W'] = dados['xmax']-dados['xmin']
  dados['H'] = dados['ymax']-dados['ymin']
  dados['Norm(xMin)'] = (dados['xmin']+(dados['W']/2))/im_w
  dados['Norm(yMin)'] = (dados['ymin']+(dados['H']/2))/im_h
  dados['Norm(W)'] = (dados['W'])/im_w
  dados['Norm(H)'] = (dados['H'])/im_h
  dados = dados.drop(['xmin',	'ymin',	'xmax',	'ymax', 'H','W'], axis = 1)

 
  textfile = open(save_path+(os.listdir(path_img)[i])[:-4]+".txt", "w")
  for j in range(len(dados)):
      lista = dados.iloc[j].to_list()
      lista[0] = int(lista[0])
      listToStr = ' '.join(map(str, lista))
      textfile.write(listToStr + "\n")

  textfile.close()
  cv2.imwrite(save_path+os.listdir(path_img)[i], frame)

print(len(os.listdir(save_path)))

Auto anotação de frames provenientes de vídeos

In [ ]:
#Setando o caminho onde o vídeo está localizado
path_video = '/content/drive/MyDrive/video/v2.mp4'

#Caminho onde os arquivos serão salvos
save_path = '/content/drive/MyDrive/imagens_auto_anotadas/'

try:
  os.mkdir( '/content/drive/MyDrive/imagens_auto_anotadas/')
  print('Criando o diretório')
except:
  print('Diretório encontrado')

#Carregando o vídeo 
cap = cv2.VideoCapture(path_video)

#Variáveis para se utilizar no salvamento dos frames e anotações
num_frame = 0
name_frame = path_vid.rsplit('/',1)[1].rsplit('.',1)[0]+'_frame_'

#Flag que vai controlar a taxa de frames que serão processados 
frame_count = 0

ret = True

#Rodando o loop equanto o vídeo não estiver finalizado 
while ret==True:

  #Lendo cada Frame
  ret, frame = cap.read()
  
  #Para quando o vídeo acabar


  #Vai pegar um vídeo a cada 10 frames
  frame_count+=1
  if frame_count ==3:
    #Variáveis para salvamento
    num_frame +=1 
    save_name = name_frame+str(num_frame)

    frame_count = 0

    if ret == True:
        #Converten do tipo numpy array para PIL.image (aumenta a precisão por algum motivo)
        frame_pil = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_pil = Image.fromarray(frame_pil)

        #Enviando a imagem para a rede
        results = model(frame_pil,  size=1280)
        
        #Transformando as predições do modelo em formato pandas DataFrame 
        dados = results.pandas().xyxy[0]
        
        #Realizando o préprocessamento necessário para transformar as predições no 
        #formato padrão das anotações do modelo YOLO 
        im_w, im_h = im.size
        dados = dados[['class','xmin', 'ymin', 'xmax', 'ymax']]
        dados['W'] = dados['xmax']-dados['xmin']
        dados['H'] = dados['ymax']-dados['ymin']
        dados['Norm(xMin)'] = (dados['xmin']+(dados['W']/2))/im_w
        dados['Norm(yMin)'] = (dados['ymin']+(dados['H']/2))/im_h
        dados['Norm(W)'] = (dados['W'])/im_w
        dados['Norm(H)'] = (dados['H'])/im_h
        dados = dados.drop(['xmin',	'ymin',	'xmax',	'ymax', 'H','W'], axis = 1)

        #Transformando o Dataframe em txt
        textfile = open(save_path+save_name+".txt", "w")
        for i in range(len(dados)):
            lista = dados.iloc[i].to_list()
            lista[0] = int(lista[0])
            listToStr = ' '.join(map(str, lista))
            textfile.write(listToStr + "\n")

        textfile.close()
        #Salvando a imagem
        cv2.imwrite(save_path+save_name+".jpg", frame)
